# Практика 1

В этом задании необходимо будет разобраться в jupyter-ноутбуке по темам первой лекции, запустить / дописать пару строк кода и ответить на несложные вопросы. Цель ноутбука - показать базовые принципы работы с текстовыми эмбеддингами, построением задачи языкового моделирования и проблемой борьбы за уменьшение количество словаря / параметров в нейронных сетях.

Также, этот код будет полезен в качестве основы для обучения собственных персональных ассистентов в рамках дополнительных заданий. 

Откуда брать ноутбук: https://github.com/dmkalash/mailru_llm_course/tree/main/Лекция%201.%20Введение%20в%20NLP

Стоит учитывать, что в некоторых частях данного ноутбука я заведомо пожертвовал оптимальностью ради большей прозрачности кода и лучшего понимания базовых принципов работы с эмбеддингами и языковым моделированием. Как оптимизировать те или иные моменты, мы будем рассказывать в следующих лекциях и семинарах.

## Fasttext

Посмотрим, как работает векторная арифметика в подходе fasttext

In [1]:
import re
import pickle 
from itertools import chain
from datetime import datetime
from collections import defaultdict

from typing import List, Dict, Optional, Iterable, Tuple

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from datasets import load_dataset
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import tokenizers
from tokenizers import Tokenizer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.processors import TemplateProcessing

2024-02-22 09:50:21.360652: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 09:50:21.360753: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 09:50:21.631369: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Датасет, с которым будем работать. 

Берем небольшое количество обучающих примеров, чтобы эксперименты можно было проводить быстро. 

In [2]:
# ds_name_2 = 'IlyaGusev/stihi_ru'

def get_dataset(train_size: int,
                test_size: int,
                ds_name_1: str = 'IlyaGusev/gazeta',
               ): 
    
    train_dataset = load_dataset(ds_name_1, split='train')
    test_dataset = load_dataset(ds_name_1, split='test')

    train_df = pd.DataFrame(train_dataset).iloc[:train_size]
    print(train_df.shape)

    test_df = pd.DataFrame(test_dataset)[:test_size]
    print(test_df.shape)

    train_texts = (train_df['title'] + '\n' + train_df['text']).tolist()
    test_texts = (test_df['title'] + '\n' + test_df['text']).tolist()
    
    return train_texts, test_texts

In [3]:
train_texts, test_texts = get_dataset(5000, 500)
all_texts = train_texts + test_texts

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/60964 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6793 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6369 [00:00<?, ? examples/s]

Dataset gazeta_dataset downloaded and prepared to /root/.cache/huggingface/datasets/IlyaGusev___gazeta_dataset/default/2.0.0/e2d171980aa248bc22e0af4f8485ad69071fc8e5f3d54a253c71eb434f6694bd. Subsequent calls will reuse this data.
(5000, 5)
(500, 5)


## Fasttext

Посмотрим, как получать эмбеддинги текстов с помощью fasttext, и как находить похожие тексты. 

Эту часть задания советую выполнять локально на ноутбуке. В Kaggle-ноутбуке из-за интернета предобученный fasttext загружается слишком медленно

In [4]:
# !pip install fasttext==0.9.2

import fasttext.util

In [5]:
fasttext.util.download_model('ru', if_exists='ignore')  # Russian

'cc.ru.300.bin'

In [6]:
ft = fasttext.load_model('cc.ru.300.bin')

In [7]:
anchor_ind = 120
print(train_texts[anchor_ind][:200], '\n')
print(train_texts[anchor_ind + 3][:200], '\n')

«Чикаго» в шаге от Кубка Стэнли

После двух поражений в Чикаго «летчикам» на своей площадке удалось отыграться, так что пятого матча все ждали с нетерпением. Его победитель заметно повышал свои шансы н 



«Будут бить, если что»

В ночь на понедельник резко обострилась ситуация в Кадашах, где уже третью неделю идет акция в защиту исторического наследия города. Как сообщил координатор «Московского совета» 




Для нахождения похожих текстов по векторам как правило используют косинусную близость, ее можно реализовать самому, можно воспользоваться готовой реализацией

In [8]:
v1 = ft.get_sentence_vector(train_texts[anchor_ind].replace('\n', ' '))
v2 = ft.get_sentence_vector(train_texts[anchor_ind + 3].replace('\n', ' '))

# K(X, Y) = <X, Y> / (||X||*||Y||)

print( (v1 * v2).sum() / (v1 ** 2).sum() ** 0.5 / (v2 ** 2).sum() ** 0.5 )
print( cosine_similarity(v1.reshape(1, -1), v2.reshape(1, -1))[0][0] )

0.9070907876494414

0.9070907


#### >>> Задание 1

Найдите в корпусе тренировочных текстов train_texts текст, наиболее похожий по косинусному расстоянию на train_texts[anchor_ind], где anchor_ind=120. В качестве ответа выведите похожесть данных текстов (число напротив "Похожесть:") 

Для всех текстов используйте единственную нормализацию как в примере выше: .replace('\n', ' ')

In [ ]:
anchor_ind = 120
need_length = 300

v1 = ft.get_sentence_vector(train_texts[anchor_ind][:need_length].replace('\n', ' '))

max_similarity = ...
max_sim_text = ...


print(
    'Номер текста', max_ind, '\n',
    'Похожесть: ', round(max_similarity, 3), '\n',
    'Текст: ', max_sim_text[:need_length]
)

В качестве ответа укажите выведенную похожесть в коде выше с той же точностью. Например: 0.395

Как мы видим, оба текста про спорт, пусть и содержат различную семантику. Если взять anchor_ind=12, то можно обнаружить в датасете дубликат - этого стоит избегать при обучении моделей

## LM на основе n-грамм

### 1 этап - предобработка

В качестве предобработки сделаем следующее:
* приведем все тексты к нижнему регистру
* унифицируем все пробельные символы

Напишите код для приведения всех слов в тексте к нижнему регистру в методе text_preprocess ниже

### 2 этап - токенизация, составление словаря

Выделим все слова в корпусе с помощью регекса, а также добавим туда базовые знаки пунктуации и пробел

In [6]:
class Tokenizer:
    def __init__(self,
                 token_pattern: str = '\w+|[\!\?\,\.\-\:]',
                 eos_token: str = '<EOS>',
                 pad_token: str = '<PAD>',
                 unk_token: str = '<UNK>'):
        self.token_pattern = token_pattern
        self.eos_token = eos_token
        self.pad_token = pad_token
        self.unk_token = unk_token
        
        self.special_tokens = [self.eos_token, self.pad_token, self.unk_token]
        self.vocab = None
        self.inverse_vocab = None
    
    def text_preprocess(self, input_text: str) -> str:
        """ Предобрабатываем один текст """
        input_text = ... # приведение к нижнему регистру
        input_text = re.sub('\s+', ' ', input_text) # унифицируем пробелы
        input_text = input_text.strip()
        return input_text
    
    def build_vocab(self, corpus: List[str]) -> None:
        assert len(corpus)
        all_tokens = set()
        for text in corpus:
            all_tokens |= set(self._tokenize(text, append_eos_token=False))
        self.vocab = {elem: ind for ind, elem in enumerate(all_tokens)}
        special_tokens = [self.eos_token, self.unk_token, self.pad_token]
        for token in special_tokens:
            self.vocab[token] = len(self.vocab) + 1
        self.inverse_vocab = {ind: elem for elem, ind in self.vocab.items()}
        return self
        
    def _tokenize(self, text: str, append_eos_token: bool = True) -> List[str]:
        text = self.text_preprocess(text)
        tokens = re.findall(self.token_pattern, text)
        if append_eos_token:
            tokens.append(self.eos_token)
        return tokens
    
    def encode(self, text: str, append_eos_token: bool = True) -> List[str]:
        """ Токенизируем текст """
        tokens = self._tokenize(text, append_eos_token)
        ids = [self.vocab.get(token, self.vocab[self.unk_token]) for token in tokens]
        return ids
    
    def decode(self, input_ids: Iterable[int], remove_special_tokens: bool = False) -> str:
        assert len(input_ids)
        assert max(input_ids) < len(self.vocab) and min(input_ids) >= 0
        tokens = []
        for ind in input_ids:
            token = self.inverse_vocab[ind]
            if remove_special_tokens and token in self.special_tokens:
                continue
            tokens.append(token)
        text = ' '.join( tokens )
        return text

In [7]:
tokenizer = Tokenizer().build_vocab(['вот такие прироги и ничего больше', '! ? . '])

In [8]:
tokenizer.vocab

{'прироги': 0,
 '!': 1,
 '?': 2,
 '.': 3,
 'и': 4,
 'больше': 5,
 'ничего': 6,
 'вот': 7,
 'такие': 8,
 '<EOS>': 10,
 '<UNK>': 11,
 '<PAD>': 12}

In [ ]:
text = 'вот такие пироги!'

In [9]:
tokenizer.encode(text)

[7, 8, 11, 1, 10]

In [10]:
tokenizer._tokenize(text, append_eos_token=False)

['вот', 'такие', 'пироги', '!']

In [11]:
tokenizer.decode(tokenizer.encode(text), remove_special_tokens=False)

'вот такие <UNK> ! <EOS>'

In [12]:
tokenizer.decode(tokenizer.encode(text), remove_special_tokens=True)

'вот такие !'

### 3 этап - LM на основе n-грамм

Напишем класс, который будет делать языковое моделирование на основе n-грамм. Изучите код, посмотрите какие функции здесь за что отвечают.

На лекции обсуждали сглаживание Лапласа для этой модели. В методе _ get_next_token определяются статистики для рассматриваемых n-грамм. В этом коде отсутствует сглаживание Лапласа. Добавьте его, согласно формуле из презентации.

К слову, если этого не сделать, то генерация будет завершаться с ошибкой - тк статистики по некоторым текстам нет, будем делить на ноль.

In [13]:
np.random.seed(42)


class StatLM:
    def __init__(self, 
                 #vocab: Dict[str, int], 
                 tokenizer: Tokenizer,
                 context_size: int = 2,
                 alpha: float = 0.1,
                 sample_top_p: Optional[float] = None
                ):
        
        assert context_size >= 2
        assert sample_top_p is None or 0.0 < sample_top_p <= 1.0
        
        self.context_size = context_size
        self.tokenizer = tokenizer
        self.alpha = alpha
        self.sample_top_p = sample_top_p
        
        self.n_gramms_stat = defaultdict(int)
        self.nx_gramms_stat = defaultdict(int)
        
    def get_token_by_ind(self, ind: int) -> str:
        return self.tokenizer.vocab.get(ind)
    
    def get_ind_by_token(self, token: str) -> int:
        return self.tokenizer.inverse_vocab.get(token, self.tokenizer.inverse_vocab[self.unk_token])
        
    #def train(self, train_token_indices: List[List[int]]):
    def train(self, train_texts: List[str]):
        for sentence in tqdm(train_texts, desc='train lines'):
            sentence_ind = self.tokenizer.encode(sentence)
            for i in range(len(sentence_ind) - self.context_size):
                
                seq = tuple(sentence_ind[i: i + self.context_size - 1])
                self.n_gramms_stat[seq] += 1
                
                seq_x = tuple(sentence_ind[i: i + self.context_size])
                self.nx_gramms_stat[seq_x] += 1
                
            seq = tuple(sentence_ind[len(sentence_ind) - self.context_size:])
            self.n_gramms_stat[seq] += 1
            
    def sample_token(self, token_distribution: np.ndarray) -> int:
        if self.sample_top_p is None:
            return token_distribution.argmax()
        else:
            token_distribution = sorted(list(zip(token_distribution, np.arange(len(token_distribution)))))
            total_proba = 0.0
            tokens_to_sample = []
            tokens_probas = []
            for token_proba, ind in sorted(token_distribution, reverse=True):
                tokens_to_sample.append(ind)
                tokens_probas.append(token_proba)
                total_proba += token_proba
                if total_proba >= self.sample_top_p:
                    break
            # для простоты отнормируем вероятности, чтобы суммировались в единицу
            tokens_probas = np.array(tokens_probas)
            tokens_probas = tokens_probas / tokens_probas.sum()
            return np.random.choice(tokens_to_sample, p=tokens_probas)
        
    def get_stat(self) -> Dict[str, Dict]:
        
        n_token_stat, nx_token_stat = {}, {}
        for token_inds, count in self.n_gramms_stat.items():
            n_token_stat[self.tokenizer.decode(token_inds)] = count
        
        for token_inds, count in self.nx_gramms_stat.items():
            nx_token_stat[self.tokenizer.decode(token_inds)] = count
        
        return {
            'n gramms stat': self.n_gramms_stat,
            'n+1 gramms stat': self.nx_gramms_stat,
            'n tokens stat': n_token_stat,
            'n+1 tokens stat': nx_token_stat,
        }
    
    def _get_next_token(self, tokens: List[int]) -> (int, str):
        denominator = self.n_gramms_stat.get(tuple(tokens), 0) + ... # TODO: сглаживание Лапласа
        numerators = []
        for ind in self.tokenizer.inverse_vocab:
            numerators.append(self.nx_gramms_stat.get(tuple(tokens + [ind]), 0) + ...) # TODO: сглаживание Лапласа
        
        token_distribution = np.array(numerators) / denominator
        max_proba_ind = self.sample_token(token_distribution)
        
        next_token = self.tokenizer.inverse_vocab[max_proba_ind]
        
        return max_proba_ind, next_token
            
    def generate_token(self, text: str, remove_special_tokens: bool = False) -> Dict:
        tokens = self.tokenizer.encode(text, append_eos_token=False)
        tokens = tokens[-self.context_size + 1:]
        
        max_proba_ind, next_token = self._get_next_token(tokens)
        
        return {
            'next_token': next_token,
            'next_token_num': max_proba_ind,
        }
    
    
    def generate_text(self, text: str, max_tokens: int, remove_special_tokens: bool = False) -> Dict:
        all_tokens = self.tokenizer.encode(text, append_eos_token=False)
        tokens = all_tokens[-self.context_size + 1:]
        
        next_token = None
        while next_token != self.tokenizer.eos_token and len(all_tokens) < max_tokens:
            max_proba_ind, next_token = self._get_next_token(tokens)
            all_tokens.append(max_proba_ind)
            tokens = all_tokens[-self.context_size + 1:]
        
        new_text = self.tokenizer.decode(all_tokens, remove_special_tokens)
        
        finish_reason = 'max tokens'
        if all_tokens[-1] == self.tokenizer.vocab[self.tokenizer.eos_token]:
            finish_reason = 'end of text'
        
        return {
            'all_tokens': all_tokens,
            'total_text': new_text,
            'finish_reason': finish_reason
        }

Для демонстрации того, что происходит, возьмем несколько коротких цитат Джейсона Стэтхема отсюда:

https://dzen.ru/a/ZRFaGN_gKhX6xTWW

In [14]:
texts = [
    'Взял нож - режь, взял дошик - ешь.',
    'Никогда не сдавайтесь, идите к своей цели! А если будет сложно – сдавайтесь.',
    'Запомни: всего одна ошибка – и ты ошибся.',
    'В жизни всегда есть две дороги: одна — первая, а другая — вторая.',
    'Делай, как надо. Как не надо, не делай.',
    'Работа не волк. Никто не волк. Только волк волк.',
    'Работа не волк. Работа - ворк. А волк - это ходить.',
    'Работа',
    ]

train_texts = texts[:-1]
test_text = texts[-1]

In [15]:
tokenizer = Tokenizer().build_vocab(train_texts)

In [16]:
tokenizer.vocab

{'не': 0,
 'идите': 1,
 'сложно': 2,
 'есть': 3,
 ',': 4,
 '-': 5,
 'никто': 6,
 'работа': 7,
 'это': 8,
 ':': 9,
 'если': 10,
 'запомни': 11,
 'и': 12,
 'ошибся': 13,
 'ходить': 14,
 'волк': 15,
 'первая': 16,
 'в': 17,
 'будет': 18,
 'цели': 19,
 'жизни': 20,
 'другая': 21,
 'всего': 22,
 'никогда': 23,
 '!': 24,
 'две': 25,
 'всегда': 26,
 'своей': 27,
 'ешь': 28,
 'взял': 29,
 'дороги': 30,
 'сдавайтесь': 31,
 'а': 32,
 'ворк': 33,
 'надо': 34,
 'нож': 35,
 '.': 36,
 'вторая': 37,
 'только': 38,
 'ты': 39,
 'режь': 40,
 'дошик': 41,
 'как': 42,
 'одна': 43,
 'делай': 44,
 'к': 45,
 'ошибка': 46,
 '<EOS>': 48,
 '<UNK>': 49,
 '<PAD>': 50}

In [17]:
# класс, который позволяем строить и использовать языковую модель на основе n-грамм
stat_lm = StatLM(tokenizer, context_size=2, alpha=0.1, sample_top_p = None)

# "обучаем" модель - считаем статистики
stat_lm.train(train_texts)

train lines:   0%|          | 0/7 [00:00<?, ?it/s]

In [18]:
# можем посмотреть статистики для n-грамм
tokens_stat = stat_lm.get_stat()
print(tokens_stat.keys())
tokens_stat['n+1 tokens stat']

dict_keys(['n gramms stat', 'n+1 gramms stat', 'n tokens stat', 'n+1 tokens stat'])


{'взял нож': 1,
 'нож -': 1,
 '- режь': 1,
 'режь ,': 1,
 ', взял': 1,
 'взял дошик': 1,
 'дошик -': 1,
 '- ешь': 1,
 'ешь .': 1,
 'никогда не': 1,
 'не сдавайтесь': 1,
 'сдавайтесь ,': 1,
 ', идите': 1,
 'идите к': 1,
 'к своей': 1,
 'своей цели': 1,
 'цели !': 1,
 '! а': 1,
 'а если': 1,
 'если будет': 1,
 'будет сложно': 1,
 'сложно сдавайтесь': 1,
 'сдавайтесь .': 1,
 'запомни :': 1,
 ': всего': 1,
 'всего одна': 1,
 'одна ошибка': 1,
 'ошибка и': 1,
 'и ты': 1,
 'ты ошибся': 1,
 'ошибся .': 1,
 'в жизни': 1,
 'жизни всегда': 1,
 'всегда есть': 1,
 'есть две': 1,
 'две дороги': 1,
 'дороги :': 1,
 ': одна': 1,
 'одна первая': 1,
 'первая ,': 1,
 ', а': 1,
 'а другая': 1,
 'другая вторая': 1,
 'вторая .': 1,
 'делай ,': 1,
 ', как': 1,
 'как надо': 1,
 'надо .': 1,
 '. как': 1,
 'как не': 1,
 'не надо': 1,
 'надо ,': 1,
 ', не': 1,
 'не делай': 1,
 'делай .': 1,
 'работа не': 2,
 'не волк': 3,
 'волк .': 4,
 '. никто': 1,
 'никто не': 1,
 '. только': 1,
 'только волк': 1,
 'волк вол

In [19]:
print('Вход:', test_text)
print('Новый токен:', stat_lm.generate_token(test_text)['next_token'])

Вход: Работа
Новый токен: не


In [20]:
print(f'Вход: "{test_text}"')
generated = stat_lm.generate_text(test_text, max_tokens=32)
print(f"Продолженный текст: \"{generated['total_text']}\"")
print(f"Причина остановки генерации: \"{generated['finish_reason']}\"" )

Вход: "Работа"
Продолженный текст: "работа не волк . никто не волк . никто не волк . никто не волк . никто не волк . никто не волк . никто не волк . никто не волк ."
Причина остановки генерации: "max tokens"


Как мы видим, модель зациклилась, и начала повторять один и тот же наиболее вероятный кусок текста.
Это нередкая проблема для всех генеративных моделей, включая современные LLM.

Для борьбы с ней делают две вещи - нормальный трейн-датасет на этапе трейна, и определенные параметры генерации
на этапе инференса (например, занизить вероятности в предсказанном распределении у тех токенов, что мы недавно уже генерировали).

Также можно использовать другие стратегии семплирования, про которые мы поговорим позднее. Например, sample_top_p в реализации класса позволяет семплировать из наиболее вероятных токенов, вероятности которых суммируются в заданное число p. Можете попробовать эту стратегию, правда, возможно ее нужно будет немного подебажить

### "Обучим" LM на n-граммах на полноценном большом датасете

In [21]:
%%time
tokenizer = Tokenizer().build_vocab(all_texts)

CPU times: user 4.78 s, sys: 7.6 ms, total: 4.79 s
Wall time: 4.79 s


In [22]:
len(tokenizer.vocab)

179367

Тут размер словаря посерьезнее

#### >>>> Задание 2: Оцените, насколько возрастет словарь, если убрать один из этапов предобработки - не приводить все тексты к нижнему регистру

В качестве ответа используйте отношение размера без предобработка к размеру с предобработкой, округленное как в коде ниже. Например: 15.305

In [ ]:
preprocessed_size = ...
without_preprocess_size = ...

print( round(without_preprocess_size / preprocessed_size) )

In [28]:
with open('tokenizer.pkl', 'wb') as fout:
    pickle.dump(tokenizer, fout)

In [39]:
train_texts, test_texts = get_dataset(5000, 500)

(5000, 5)
(500, 5)


In [40]:
%%time
stat_lm = StatLM(tokenizer, context_size=2, alpha=0.1, sample_top_p = None)
stat_lm.train(train_texts)

train lines:   0%|          | 0/5000 [00:00<?, ?it/s]

CPU times: user 11.7 s, sys: 150 ms, total: 11.9 s
Wall time: 11.8 s


In [41]:
for text in test_texts[:3]:
    title = text.split('\n')[0]
    generated = stat_lm.generate_text(title, max_tokens=128)
    print(generated['total_text'], '\n')

в германии объяснили упоминание имени путина на протестах в берлине . в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в 

делегации израиля и сша прибыли в оаэ для обсуждения соглашения о сотрудничестве с . в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том , что в том 

оппозиция белоруссии объявила о создании новой партии вместе с . в том , что в том , что в том , что в том , что в том , что в том , что в том , что 

In [42]:
%%time
stat_lm = StatLM(tokenizer, context_size=4, alpha=0.1, sample_top_p = None)
stat_lm.train(train_texts)

train lines:   0%|          | 0/5000 [00:00<?, ?it/s]

CPU times: user 12.7 s, sys: 427 ms, total: 13.2 s
Wall time: 13.1 s


In [43]:
for text in test_texts[:3]:
    title = text.split('\n')[0]
    generated = stat_lm.generate_text(title, max_tokens=128)
    print(generated['total_text'], '\n')

в германии объяснили упоминание имени путина на протестах в берлине рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль рауль 

делегации израиля и сша прибыли в оаэ для обсуждения соглашения о сотрудничестве , заключенного 25 ноября 2008 года в разных банках было создано более сотни фиктивных счетов для отмывания денег . чтобы не встретить серьезн

Попробуйте перебрать параметры в поисках лучшего качества модели: context_size, alpha, и стратегию генерации sample_top_p=0.9 (возможно, эту стратегию придется немного подебажить)

## LM на основе RNN

Рекуррентные сети формально могут обрабатывать последовательность произвольной длины - 
можем просто через RNN-ячейку рекуррентно пропускать последовательность, а в конце посчитать градиент
по этой большой цепочке.

На практике все равно есть ограничения:
1. Градиент по большой последовательности очень долго считать - с помощью backpropagation through time (BPTT) для каждого элемента последовательности придется считать градиент
2. Для ускорения обучения и для лучшей сходимости тренировочные примеры (то есть, последовательности = тексты) объединяют во время обучения в батчи. Чтобы батч представить в виде тензора, все последовательности нужно выравнить (мы же не можем в матрице сделать длины строк разными). Для этого используются последовательности с фиксированной длиной, называемой длиной контекста.

Теперь давайте построим LM с помощью рекуррентной сети

Попробуем использовать tokenizer из прошлого раздела, уже обученный на словах 

In [44]:
with open('tokenizer.pkl', 'rb') as fin:
    tokenizer = pickle.load(fin)

In [45]:
len(tokenizer.vocab)

179367

Составим датасеты в torch-формате

In [46]:
class NewsDataset(torch.utils.data.Dataset):
    
    def __init__(self, inputs: List[List[int]], targets: Optional[List[List[int]]] = None):
        self.inputs = torch.LongTensor(inputs)
        self.targets = None
        if targets is not None:
            self.targets = torch.LongTensor(targets)
        
    def __len__(self):
        return self.inputs.shape[0]

    def __getitem__(self, idx: int) -> (List[str], int):
        if self.targets is None:
            return self.inputs[idx]
        else:
            return self.inputs[idx], self.targets[idx]

In [47]:
context_size = 32

def get_tokenized_data(tokenizer: Tokenizer, 
                       texts: List[str], 
                       context_size: int) -> (List[List[int]], List[List[int]]):
    tokenized_inputs, tokenized_targets = [], []
    for text in tqdm(texts):
        tokens = tokenizer.encode(text, append_eos_token=True)
        for i in range(len(tokens) - context_size):
            inputs = tokens[i: i + context_size]
            targets = tokens[i + 1: i + 1 + context_size]
            tokenized_inputs.append(inputs)
            tokenized_targets.append(targets)
    return tokenized_inputs, tokenized_targets

Возьмем для начала малое количество текстов, просто чтобы отладить процесс

In [48]:
len(train_texts), len(test_texts)

(5000, 500)

In [49]:
batch_size = 32

tokenized_inputs_train, tokenized_targets_train = get_tokenized_data(tokenizer, train_texts[:120], context_size)
tokenized_inputs_test, tokenized_targets_test = get_tokenized_data(tokenizer, test_texts[:60], context_size)

train_dataset = NewsDataset(tokenized_inputs_train, tokenized_targets_train)
test_dataset = NewsDataset(tokenized_inputs_test, tokenized_targets_test)

train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)

t0, t1 = next(iter(train_dl))
t0.shape, t1.shape

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

(torch.Size([32, 32]), torch.Size([32, 32]))

Параметры обучения и модели

In [50]:
optimizer_params = {}

model_params = {
    'vocab_size': len(tokenizer.vocab),
    'embed_dim': 300,
    'hidden_size': 64
}

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

n_epochs = 10

Архитектура сети представлена ниже. 

Здесь мы каждому токену из последовательности сопоставляем обучаемый вектор с помощью слоя nn.Embedding, и всего таких векторов слой знает vocab_size штук, где vocab_size - размер словаря.

Каждый такой обучаемый вектор имеет размерность embed_dim.

После того, как мы получим "умное" представление всей последовательности - вектор размерности hidden_size, мы хотим отобразить его в пространство словаря, и это отображение (вектор) мы будем использовать как логиты, из которых получается вероятностное распределение для следующего токена. За это отображение отвечает линейный слой.

Заполните параметры nn.Embedding и nn.Linear с учетом написанного выше

In [51]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

class RecLM(nn.Module):
    def __init__(self, vocab_size: int, embed_dim: int, hidden_size: int):
        super(RecLM, self).__init__()
        self.embed = nn.Embedding(..., ...) # TODO заполнить размерности
        self.rnn = nn.RNNCell(embed_dim, hidden_size, nonlinearity='tanh', bias=True)
        
        self.linear = nn.Linear(..., ...)  # TODO заполнить размерности
        # иногда вместо этого используют ту же матрицу, что в nn.Embedding 
        
    def forward(self, inputs: torch.LongTensor):
        """
        inputs: int, bs x seq
        # pad_mask: bool, bs x seq
        """
        embed = self.embed(inputs) # bs x seq x dim
        h_n = None
        outputs = []
        for seq_elem in embed.transpose(0, 1):
            if h_n is None:
                h_n = self.rnn(seq_elem) # bs x out_dim
            else:
                h_n = self.rnn(seq_elem, h_n)
            outputs.append(h_n)
        
        outputs = torch.stack(outputs).transpose(0, 1) # bs x seq x dim
        return self.linear(outputs) # bs x seq x vocab_size

In [52]:
model = RecLM(vocab_size=model_params['vocab_size'], 
              embed_dim=model_params['embed_dim'], 
              hidden_size=model_params['hidden_size'])

В качестве лосса используем кросс-энтропию - ведь по сути мы решаем задачу многоклассовой классификации, где классы - это токены из словаря. Почитать про лосс можно ниже:

nn.CrossEntropyLoss: https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html

Напишем класс Trainer , с помощью которого мы будем обучать нейронку. Изучите, что происходит в коде

In [102]:
class Trainer:
    
    def __init__(
        self, 
        model_params: dict,
        optimizer_params: dict, 
        n_epochs: int,
        train_dataloader: torch.utils.data.DataLoader, 
        val_dataloader: torch.utils.data.DataLoader,
        device: str
    ):

        self.model = RecLM(**model_params)
        self.device = torch.device(device)
        self.model.to(self.device)
        self.optimizer = torch.optim.AdamW(params=self.model.parameters(), **optimizer_params)
        
        ts = datetime.strftime(datetime.today(), '%Y-%m-%d-%H-%M')
        LOG_DIR = f'./checkpoints/{ts}'
        
        hparams = chain(model_params.items(), optimizer_params.items())

        for k, v in hparams:
            LOG_DIR += f'-{k}-{v}'
        
        self.writer = SummaryWriter(log_dir=LOG_DIR)
        self.checkpoint_path = LOG_DIR

        self.best_val_loss = float('inf')
        self.n_epochs = n_epochs
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        
        self.loss_fn = torch.nn.CrossEntropyLoss(reduction = 'none')

    def iterate_over_dataloader(self, dataloader, suffix: str, epoch: int, update_weights=False):
        
        if update_weights:
            self.model.train()
        else:
            self.model.eval()
        
        loss_over_epoch = 0
        num_batches = 0

        for batch in tqdm(dataloader, desc='batches'):

            if update_weights:
                self.optimizer.zero_grad()

            tokens, labels = batch
            
            tokens = tokens.to(self.device)
            labels = labels.to(self.device)
            
            logits = self.model(tokens) # bs x seq x vocab_size
            
            loss_values = self.loss_fn(logits.transpose(1, 2), labels) # N x C x seq_len
            loss_value = loss_values.mean()
            
            if update_weights:
                loss_value.backward()
                self.optimizer.step()
            
            loss_item = loss_value.item()
            loss_over_epoch += loss_item
            num_batches += 1
        
        avg_loss = loss_over_epoch / num_batches
        print(f'Epoch {epoch} loss for {suffix}: {avg_loss}')
        self.writer.add_scalar(tag=f'Loss/{suffix}', scalar_value=avg_loss, global_step=epoch)

        if avg_loss < self.best_val_loss:
            self.best_val_loss = loss_item
            torch.save(self.model.state_dict(), os.path.join(self.checkpoint_path, 'rnn_lm.pt'))

            
    def train_model(self):
        for epoch_num in tqdm(range(self.n_epochs)):
            self.iterate_over_dataloader(
                dataloader=self.train_dataloader, suffix='train', epoch=epoch_num, update_weights=True
            )
            self.iterate_over_dataloader(
                dataloader=self.val_dataloader, suffix='val', epoch=epoch_num
            )

И прежде чем обучать, давайте посмотрим, насколько большая у нас получилась модель.

In [54]:
def get_nn_params_stat(model: nn.Module) -> None:
    
    def iter_mul(inputs: Iterable) -> int:
        mul = 1
        for elem in inputs:
            mul *= elem
        return mul
    
    shapes = [p.shape for p in model.parameters()]
    for p_shape in shapes:
        print(p_shape)
    total_count = sum([iter_mul(p_shape) for p_shape in shapes])
    print('Total params:', total_count)
    print('Model param size in Mb:', total_count * 4 / (2 ** 20))

torch.Size([179367, 300])
torch.Size([64, 300])
torch.Size([64, 64])
torch.Size([64])
torch.Size([64])
torch.Size([179367, 64])
torch.Size([179367])
Total params: 65492379
Model param size in Mb: 249.83359909057617


In [55]:
get_nn_params_stat(model)

torch.Size([179367, 300])
torch.Size([64, 300])
torch.Size([64, 64])
torch.Size([64])
torch.Size([64])
torch.Size([179367, 64])
torch.Size([179367])
Total params: 65492379
Model param size in Mb: 249.83359909057617


В гугл колабе такая модель на гпу не поместится, а если бы и поместилась - обучать такое пришлось бы долго.

Как мы видим, подавляющая часть параметров содержится в тензорах torch.Size([179367, 300]) и torch.Size([179367, 64]) - это параметры слоя эмбеддинга, при этом 179367 - это размерность словаря

И, как правило, во всех подобных сетях слой nn.Embedding содержит много параметров, и в разных статьях этот слой пытаются как-то "облегчить" - либо декомпозировать, либо уменьшить размер словаря. 

Мы пойдем по пути уменьшения словаря, но для этого нам нужно представить текст не словами, а более оптимальными языковыми единицами. Это достигается с помощью BPE кодирования, которое мы будем обсуджать в курсе. А пока, давайте просто им воспользуемся, после небольшого задания.

In [60]:
model = RecLM(vocab_size=model_params['vocab_size'] // 2, 
              embed_dim=model_params['embed_dim'], 
              hidden_size=model_params['hidden_size'])

get_nn_params_stat(model)

torch.Size([89683, 300])
torch.Size([64, 300])
torch.Size([64, 64])
torch.Size([64])
torch.Size([64])
torch.Size([89683, 64])
torch.Size([89683])
Total params: 32757719
Model param size in Mb: 124.96078109741211


#### >>>> Задание 3

посмотрите, во сколько раз уменьшиться размер модели (в мегабайтах), если уменьшить в два раза размер словаря.  Пример: 5.423

In [ ]:
round( большее / меньшее, 3 )

#### >>>> Задание 4

Допустим, мы знаем, что на нашей гпу-карте мы можем выделить только 15 мб для нашей модели (при этом, все остальные ресурсы под оптимизатор, датасет и прочее уже учтены). В какое минимальное целое кол-во раз нужно уменьшить размер словаря, чтобы выполнить это требование?

In [ ]:
max_drop = ...

model = RecLM(vocab_size=model_params['vocab_size'] // max_drop,
              embed_dim=model_params['embed_dim'], 
              hidden_size=model_params['hidden_size'])

get_nn_params_stat(model)
print(f'Ответ: {max_drop}')

Ответ - целое число, например 10

### Уменьшим словарь

Обучим BPE-токенизатор на нашем корпусе. Суть несложная - в качестве токенов мы будем использовать последовательности символов, которые оптимальнее всего сжимают текст (если последовательность токенов встречается очень часто - будем использовать ее как токен, и так итеративно можем построить словарь фиксированного размера). 

In [81]:
import tokenizers
from tokenizers import Tokenizer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.processors import TemplateProcessing

In [82]:
VOCAB_SIZE = 5_000

In [83]:
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()

In [84]:
tokenizer.post_processor = TemplateProcessing(
    single="[BOS] $A [EOS]",
    special_tokens=[("[BOS]", 1), ("[EOS]", 4)],
)

In [85]:
trainer = BpeTrainer(vocab_size=VOCAB_SIZE, 
                     special_tokens=["[UNK]", "[BOS]", "[SEP]", "[PAD]", "[EOS]"],
                    show_progress=True)

In [86]:
def simple_preprocess(text: str) -> str:
    return text.lower()

In [87]:
len(train_texts), len(test_texts)

(5000, 500)

In [88]:
train_texts = list(map(simple_preprocess, train_texts))
test_texts = list(map(simple_preprocess, test_texts))

In [89]:
with open('new_train.txt', 'w') as fout:
    print('\n'.join(train_texts), file=fout)
    
with open('new_test.txt', 'w') as fout:
    print('\n'.join(test_texts), file=fout)

In [90]:
%%time
files = [f'new_{key}.txt' for key in ['train', 'test']]
tokenizer.train(files, trainer)




CPU times: user 14.7 s, sys: 610 ms, total: 15.3 s
Wall time: 5.11 s


In [91]:
tokenizer.get_vocab_size()

5000

In [92]:
text = 'Вот такие пироги!'

print(tokenizer.encode(text).tokens)
print(tokenizer.encode(text.lower()).tokens)
print(tokenizer.encode(text.lower()).ids)
print(tokenizer.encode(text.lower()).special_tokens_mask)


['[BOS]', '[UNK]', 'от', 'такие', 'пи', 'ро', 'ги', '!', '[EOS]']
['[BOS]', 'вот', 'такие', 'пи', 'ро', 'ги', '!', '[EOS]']
[1, 1119, 2033, 236, 146, 234, 5, 4]
[1, 0, 0, 0, 0, 0, 0, 1]


In [93]:
{ind: token for token, ind in tokenizer.get_vocab().items()}[0]

'[UNK]'

In [94]:
tokenizer.get_vocab()['[EOS]']

4

Токенизируем трейн и тест данные

In [95]:
context_size = 32

def get_bpe_tokenized_data(tokenizer: tokenizers.Tokenizer, 
                           texts: List[str], 
                           context_size: int) -> (List[List[int]], List[List[int]]):
    tokenized_inputs, tokenized_targets = [], []
    for text in tqdm(texts):
        tokens = tokenizer.encode(text).ids
        for i in range(len(tokens) - context_size):
            inputs = tokens[i: i + context_size]
            targets = tokens[i + 1: i + 1 + context_size]
            tokenized_inputs.append(inputs)
            tokenized_targets.append(targets)
    return tokenized_inputs, tokenized_targets

In [96]:
batch_size = 32

tokenized_inputs_train, tokenized_targets_train = get_bpe_tokenized_data(tokenizer, train_texts[:1024], context_size)
tokenized_inputs_test, tokenized_targets_test = get_bpe_tokenized_data(tokenizer, test_texts[:128], context_size)

train_dataset = NewsDataset(tokenized_inputs_train, tokenized_targets_train)
test_dataset = NewsDataset(tokenized_inputs_test, tokenized_targets_test)

train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)

t0, t1 = next(iter(train_dl))
t0.shape, t1.shape

  0%|          | 0/1024 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

(torch.Size([32, 32]), torch.Size([32, 32]))

Чтобы эксперименты можно запустить на Kaggle или Google Colab, выставим небольшие параметры (размер словаря выбран по тем же принципам быстроты экспериментов, но обычно он выбирается 30_000 или в радиусе). 

Если у вас есть собственные мощности либо свободное время, можете дополнительно поэкспермиентировать с сетями помощнее и перебрать параметры.

In [97]:
optimizer_params = {}

model_params = {
    'vocab_size': tokenizer.get_vocab_size(),
    'embed_dim': 300,
    'hidden_size': 64
}

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

n_epochs = 1

In [98]:
model = RecLM(vocab_size=model_params['vocab_size'], 
              embed_dim=model_params['embed_dim'], 
              hidden_size=model_params['hidden_size'])

In [99]:
get_nn_params_stat(model)

torch.Size([5000, 300])
torch.Size([64, 300])
torch.Size([64, 64])
torch.Size([64])
torch.Size([64])
torch.Size([5000, 64])
torch.Size([5000])
Total params: 1848424
Model param size in Mb: 7.051177978515625


Можете оценить, во сколько раз изменился размер сети, при этом весь текст мы кодируем, и никакие токены не теряем (в случае токенизации по словам, выкидывание слова означало потерю сигнала и замену токена на Unknown)

Вывод: большую часть параметров подобных сетей содержит слой эмбеддинга. Во многих работах это пытаются исправить: декомпозировать этот слой низкоранговыми матрицами, переиспользовать слой эмбеддингов из начала сети в конце и др. Ну и самое главное - использовать словарь поменьше, 30_000 - классический размер словаря с BPE-токенизацией.

In [103]:
trainer = Trainer(model_params, optimizer_params, n_epochs,
       train_dl, test_dl, device)

In [104]:
import os

In [105]:
trainer.train_model()

  0%|          | 0/1 [00:00<?, ?it/s]

batches:   0%|          | 0/34677 [00:00<?, ?it/s]

Epoch 0 loss for train: 5.032392976688095


batches:   0%|          | 0/4603 [00:00<?, ?it/s]

Epoch 0 loss for val: 5.512741004076569


## Результат

С помощью обученной модели можно что-нибудь сгенерировать с помощью кода ниже. Поисследуйте поведение модели, насколько разнообразный текс генерируется, насколько текст правдоподбный

#### >>>> Задание 5.

В лекциях мы рассматривали, как получить из вектора логитов вероятностное распределение (все числа в векторе суммируются в 1, все числа от 0 до 1). Эта функция также реализована в torch, в модуле torch.nn.functional. Допишите в методе _ get_next_token только эту функцию (при ее добавлении код в ячейках ниже не должен падать), и это же слово используйте в качестве ответа. Например: relu

In [106]:
class TextGenerator:
    def __init__(self, 
                 model: nn.Module, 
                 tokenizer: tokenizers.Tokenizer,
                 context_size: int,
                 eos_token_id: int,
                 sample_top_p: Optional[float] = None):
        
        self.model = model.eval()
        self.tokenizer = tokenizer
        self.eos_token_id = eos_token_id
        self.context_size = context_size
        self.sample_top_p = sample_top_p
        
    def sample_token(self, token_distribution: np.ndarray) -> int:
        if self.sample_top_p is None:
            return token_distribution.argmax()
        else:
            token_distribution = sorted(list(zip(token_distribution, np.arange(len(token_distribution)))))
            total_proba = 0.0
            tokens_to_sample = []
            tokens_probas = []
            for token_proba, ind in sorted(token_distribution, reverse=True):
                tokens_to_sample.append(ind)
                tokens_probas.append(token_proba)
                total_proba += token_proba
                if total_proba >= self.sample_top_p:
                    break
            # для простоты отнормируем вероятности, чтобы суммировались в единицу
            tokens_probas = np.array(tokens_probas)
            tokens_probas = tokens_probas / tokens_probas.sum()
            return np.random.choice(tokens_to_sample, p=tokens_probas)
    
    
    def _get_next_token(self, tokens: List[int]) -> (int, str):
        tensor_inputs = torch.LongTensor([tokens])
        with torch.no_grad():
            logits = model(tensor_inputs)[0][-1]
            func = torch.nn.functional. ... # TODO используем функцию из torch.nn.functional, чтобы получить вероятности
            token_distribution = func(logits)
            max_proba_ind = self.sample_token(token_distribution.numpy())
        
        # print(token_distribution.shape, token_distribution)
        next_token = self.tokenizer.id_to_token(max_proba_ind)
        
        return max_proba_ind, next_token
            
    def generate_token(self, text: str, remove_special_tokens: bool = False) -> Dict:
        tokens = self.tokenizer.encode(text.lower()).ids
        if tokens[-1] == self.eos_token_id:
            tokens.pop()
        tokens = tokens[-self.context_size:]
        max_proba_ind, next_token = self._get_next_token(tokens)
        
        return {
            'next_token': next_token,
            'next_token_num': max_proba_ind,
        }
    
    
    def generate_text(self, 
                      text: str,
                      max_tokens: int, 
                      remove_special_tokens: bool = False,
                      ) -> Dict:
        
        all_tokens = tokenizer.encode(text.lower()).ids
        if all_tokens[-1] == self.eos_token_id:
            all_tokens.pop()
        tokens = all_tokens[-self.context_size:]
        if not tokens:
            return {
                'all_tokens': all_tokens,
                'total_text': '',
                'finish_reason': 'empty_input'
            }
        
        max_proba_ind = None
        while max_proba_ind != self.eos_token_id and len(all_tokens) < max_tokens:
            max_proba_ind, next_token = self._get_next_token(tokens)
            all_tokens.append(max_proba_ind)
            tokens = all_tokens[-self.context_size:]
        
        new_text = self.tokenizer.decode(all_tokens, remove_special_tokens)
        
        finish_reason = 'max tokens'
        if all_tokens[-1] == self.eos_token_id:
            finish_reason = 'end of text'
        
        return {
            'all_tokens': all_tokens,
            'total_text': new_text,
            'finish_reason': finish_reason
        }

In [107]:
text_generator = TextGenerator(model, 
                               tokenizer,
                               context_size,
                               tokenizer.token_to_id('[EOS]'), 
                               sample_top_p=None)

In [108]:
text = "Вот такие пироги"
text_generator.generate_token(text, )

/tmp/ipykernel_34/1024563228.py:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  token_distribution = func(logits)


{'next_token': 'удовлетво', 'next_token_num': 3513}

In [109]:
text = "В современном мире "
text_generator.generate_token(text, )

/tmp/ipykernel_34/1024563228.py:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  token_distribution = func(logits)


{'next_token': 'фы', 'next_token_num': 4478}

In [110]:
text = "В современном мире "
text_generator.generate_text(text, max_tokens=64)['total_text']

/tmp/ipykernel_34/1024563228.py:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  token_distribution = func(logits)


'[BOS] в совреме нном мире фы результаты а фильм кварти факту именно менять фи стол определе государство v оо стратеги жители кни писа представля вши лась заключи дца добавил цитирует компаний инициати ской важно голки шка колле goo дж 65 паде 75 25 раздо расследование хол результаты а прием премьера страте жители кни писа представля вши лась заключи дца добавил цитирует компаний инициати ской'

Модель достаточно простая - малый словарь, мало параметров к RNN, мало обучали, на малом количестве данных. Это приводит к тому, что тексты получаются не самые правдоподобные. Экспериментируя с гиперпараметрами (включая размер выбранного датасета), мне удалось достичь приемлемого качества генерации. Ради интереса, можете попробовать сделать также.